In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('housing.csv')

Elimina los máximos

In [3]:
df = df[df.median_house_value != df.median_house_value.max()]

## Crea un modelo de regresión lineal simple usando el ingreso y calcula su R2

In [4]:
model_income = linear_model.LinearRegression()
X = df[['median_income']]
target = 'median_house_value'
y = df[target]

model_income.fit(X, y)
predictions_income = model_income.predict(X)

r2_score(y_pred=predictions_income, y_true=y)

0.41294861985820197

## Crea un modelo de regresión lineal simple usando la edad de las casas

## Crea un modelo de regresión lineal multiple usando las variables anteriores

Escribe la ecuación del modelo

In [5]:
# Interepto


In [6]:
# Coeficientes


In [7]:
# Orden


$$Precio = \beta_0 + \beta_1 * x_1 + \beta_2* x_2$$

$$Precio = -10189 + 1744 * edad + 43169* ingreso$$

### Agrega la variable de households

Escribe la ecuación del modelo

### Agrega todas

In [8]:
### Verific los NA's


In [9]:
# Quita NA's 


In [10]:
# Convertir a one hot encoding 


### Visualizando el modelo, calibracion

### Compara la calibración de todos los modelos anteriores